# 技术指标计算结果验证
前置条件：从mongodb查出pd格式的蜡烛图数据

In [1]:
# 配置日志 & 加载路径 & 检查Setting清单
import sys
from pathlib import Path
import logging
import pandas as pd
from datetime import datetime

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

print("=== 路径设置测试 ===")
print(f"当前工作目录: {Path.cwd()}")
print(f"当前 Python 路径: {sys.path[:3]}...")  # 只显示前3个避免太长

# 正确的路径设置
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"\n项目根目录: {project_root}")

# 检查系统配置
from src.config.settings import config

print("=== 系统配置 ===")
print(f"特征窗口大小: {config.FEATURE_WINDOW_SIZE}")
print(f"时间窗口设置: {config.TIME_WINDOWS}")
print(f"分类阈值类别数: {len(config.CLASSIFICATION_THRESHOLDS)}")
print()

print("分类标准:")
for class_label, (min_val, max_val) in config.CLASSIFICATION_THRESHOLDS.items():
    print(f"  类别 {class_label}: {min_val}% 到 {max_val}%")

=== 路径设置测试 ===
当前工作目录: /Users/anthony/Documents/github/technial_analysis_helper/notebooks
当前 Python 路径: ['/usr/local/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/usr/local/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/usr/local/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload']...

项目根目录: /Users/anthony/Documents/github/technial_analysis_helper
=== 系统配置 ===
特征窗口大小: 300
时间窗口设置: {'short': 12, 'medium': 48, 'long': 192}
分类阈值类别数: 7

分类标准:
  类别 1: -100% 到 -5.0%
  类别 2: -5.0% 到 -2.0%
  类别 3: -2.0% 到 -0.5%
  类别 4: -0.5% 到 0.5%
  类别 5: 0.5% 到 2.0%
  类别 6: 2.0% 到 5.0%
  类别 7: 5.0% 到 100%


In [5]:
#测试获取数组格式的数据
from src.data.mongodb_handler import mongo_handler

data = mongo_handler.get_candlestick_data(inst_id="ETH-USDT-SWAP", limit=1000)
print(len(data))

from src.utils.feature_engineering import feature_engineer

data_tuple = feature_engineer.create_training_dataset(data)

from IPython.display import display
display("=== DataFrame ===", data_tuple[0])
display("=== Series ===", data_tuple[1])

2026-01-23 14:00:35,161 - src.utils.feature_engineering - INFO - Creating training dataset from 1000 records
2026-01-23 14:00:35,162 - src.utils.feature_engineering - INFO - Parameters: stride=1, prediction_horizon=24


1000


2026-01-23 14:00:41,810 - src.utils.feature_engineering - INFO - Created training dataset with 677 samples
2026-01-23 14:00:41,814 - src.utils.feature_engineering - INFO - Target distribution: {2: 155, 1: 118, 3: 112, 4: 109, 5: 103, 6: 80}


'=== DataFrame ==='

,short_rsi,short_bb_upper,short_bb_lower,short_bb_middle,short_bb_position,short_price_ma_ratio,medium_rsi,medium_bb_upper,medium_bb_lower,medium_bb_middle,...,raw_price_min,raw_price_max,raw_price_change_mean,raw_price_change_std,raw_price_change_sum,raw_volume_mean,raw_volume_std,raw_price_range_mean,raw_price_range_max,timestamp
0,NaN,NaN,NaN,NaN,NaN,NaN,62.406151,2437.723699,2411.968301,2424.8460,...,2173.81,2668.53,-0.547358,17.875358,-163.66,2.159961e+06,1.942597e+06,26.106767,178.97,1751130000000
1,NaN,NaN,NaN,NaN,NaN,NaN,56.160110,2439.212884,2411.985116,2425.5990,...,2173.81,2668.53,-0.641171,17.834927,-191.71,2.155378e+06,1.945025e+06,26.089267,178.97,1751133600000
2,NaN,NaN,NaN,NaN,NaN,NaN,52.873752,2439.901433,2412.311567,2426.1065,...,2173.81,2668.53,-0.660033,17.835319,-197.35,2.146100e+06,1.947310e+06,26.011167,178.97,1751137200000
3,NaN,NaN,NaN,NaN,NaN,NaN,50.703611,2439.847185,2413.505815,2426.6765,...,2173.81,2668.53,-0.642642,17.829743,-192.15,2.143864e+06,1.948890e+06,26.004033,178.97,1751140800000
4,NaN,NaN,NaN,NaN,NaN,NaN,55.803368,2440.525227,2414.705773,2427.6155,...,2173.81,2668.53,-0.603244,17.834095,-180.37,2.141511e+06,1.950714e+06,25.995067,178.97,1751144400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,NaN,NaN,NaN,NaN,NaN,NaN,49.617280,3765.130458,3707.991542,3736.5610,...,2951.37,3842.97,2.336388,25.982188,698.58,2.166795e+06,1.362374e+06,41.854167,128.55,1753549200000
673,NaN,NaN,NaN,NaN,NaN,NaN,54.839287,3765.016364,3710.025636,3737.5210,...,2951.37,3842.97,2.320803,25.972556,693.92,2.166885e+06,1.362323e+06,41.941033,128.55,1753552800000
674,NaN,NaN,NaN,NaN,NaN,NaN,57.907459,3766.034807,3711.550193,3738.7925,...,2951.37,3842.97,2.372910,25.973710,709.50,2.162663e+06,1.365859e+06,41.893800,128.55,1753556400000
675,NaN,NaN,NaN,NaN,NaN,NaN,55.756994,3766.274060,3713.183940,3739.7290,...,2951.37,3842.97,2.336488,25.977649,698.61,2.159383e+06,1.368491e+06,41.880900,128.55,1753560000000


'=== Series ==='

0      4
1      4
2      4
3      4
4      4
      ..
672    2
673    2
674    2
675    2
676    2
Name: target, Length: 677, dtype: int64